In [1]:
import os
import math
import json
import numpy as np
from scipy.optimize import curve_fit
import plotly
import plotly.express as px
import plotly.graph_objects as go
import glob

In [2]:
def combine_attrs(list1, list2):
    return [str(list1[i]) + ', ' + str(list2[i]) for i in range(len(list1))]

def data(json_file):
    j = json.load(open(json_file))
    settings = os.path.basename(json_file).replace('.json', '').split('_')
    j['dist'] = settings[0]
    j['uni_or_bi'] = settings[1]
    j['vocab_size'] = int(settings[2])
    j['softmax'] = True if settings[3] == 'softmax' else False
    j['settings'] = settings
    j['losses'] = []
    if 'lstm' in settings:
        j['model_type'] = 'lstm'
    elif 'ffnn' in settings:
        j['model_type'] = 'ffnn'
    else:
        j['model_type'] = 'trf'
    if 'embd256' in settings:
        j['embd_size'] = 256
    else:
        j['embd_size'] = 64
    for i in range(len(j['train_losses'])):
        j['losses'].extend(j['train_losses'][i])
    del j['train_losses']
    del j['table']
    if 'val_losses' in j:
        print(json_file)
    return j

json_files = paths = glob.glob('results/u*.json') + glob.glob('results/m*.json') + glob.glob('results/n*.json')

jsons = []
for json_file in json_files:
    try:
        d = data(json_file)
        jsons.append(d)
    except:
        print(json_file)

test_set_perplexity = []
entropy = []
dist = []
uni_or_bi = []
vocab_size = []
softmax = []
model_type = []
embd_size = []

for j in jsons:
    test_set_perplexity.append(min(j['test_set_perplexities']))
    entropy.append(j['entropy'])
    dist.append(j['dist'])
    uni_or_bi.append(j['uni_or_bi'])
    vocab_size.append(j['vocab_size'])
    softmax.append(j['softmax'])
    model_type.append(j['model_type'])
    embd_size.append(j['embd_size'])
    
for i in range(len(entropy)):
    if uni_or_bi[i] == 'bigrams':
        entropy[i] = entropy[i] / 2
    
test_set_avg_cross_entropy = [math.log(x) for x in test_set_perplexity]

df = {
    'Test set perplexity': test_set_perplexity,
    'Test set average cross-entropy': test_set_avg_cross_entropy,
    'Entropy': entropy,
    'Distribution': dist,
    'Uni- or bigram': uni_or_bi,
    'Vocab size': vocab_size,
    'Softmax': softmax,
    'Model type': model_type,
    'Embedding size': embd_size
}

In [3]:
# curve fitting
def exponential(x, a, b, c):
    return a * np.exp(b * x) + c

def linear(x, a, b):
    return a * x + b

x_ent = np.linspace(min(entropy), max(entropy), 100)

ppl_vs_ent = curve_fit(exponential, entropy, test_set_perplexity)

y_ppl = exponential(x_ent, *ppl_vs_ent[0])

ce_vs_ent = curve_fit(linear, entropy, test_set_avg_cross_entropy)

y_ce = linear(x_ent, *ce_vs_ent[0])

ppl_fit = go.Scatter(
    x=x_ent,
    y=y_ppl,
    mode='lines',
    name=f'{ppl_vs_ent[0][0]:.2f} * exp({ppl_vs_ent[0][1]:.2f} * x) + {ppl_vs_ent[0][2]:.2f}'
)

ce_fit = go.Scatter(
    x=x_ent,
    y=y_ce,
    mode='lines',
    name=f'{ce_vs_ent[0][0]:.2f} * x + {ce_vs_ent[0][1]:.2f}'
)

y_equals_x = go.Scatter(
    x=x_ent,
    y=x_ent,
    mode='lines',
    name='Theoretical limit'
)

default_colors = plotly.colors.qualitative.Plotly
next_color = default_colors[13 % len(default_colors)]
ppl_fit.update(marker_color=next_color)
ce_fit.update(marker_color=next_color)
print()

In [4]:
# R^2
residuals = test_set_avg_cross_entropy - linear(np.array(entropy), *ce_vs_ent[0])
ss_res = np.sum(residuals**2)
ss_tot = np.sum((test_set_avg_cross_entropy - np.mean(test_set_avg_cross_entropy))**2)
r_squared = 1 - (ss_res / ss_tot)
print(f'Overall R^2: {r_squared}')

Overall R^2: 0.9825107684138789


In [5]:
# plotting options
legend_dict = dict(
    orientation='h',
    y=-0.15,
)

marker_dict = dict(
    size=12,
    opacity=0.5,
)

In [6]:
fig = px.scatter(
    df,
    x='Entropy',
    y='Test set perplexity',
    title='Test set perplexity vs. entropy',
    labels={'x': 'Entropy', 'y': 'Test set perplexity', 'color': 'Uni- or bigram'},
    color=uni_or_bi,
    hover_data={'Vocab size': True, 'Softmax': True, 'Distribution': True, 'Model type': True, 'Embedding size': True},
)
fig.update_layout(legend=legend_dict)
fig.update_traces(marker=marker_dict)
fig.add_trace(ppl_fit)
fig.data = fig.data[::-1]
fig.write_html('plots/uni_or_bi.html')

In [7]:
fig = px.scatter(
    df,
    x='Entropy',
    y='Test set perplexity',
    title='Test set perplexity vs. entropy',
    labels={'x': 'Entropy', 'y': 'Test set perplexity', 'color': 'Vocab size'},
    color=[str(x) for x in vocab_size],
    category_orders={'color': reversed([str(x) for x in sorted(vocab_size)])},
    hover_data={'Uni- or bigram': True, 'Softmax': True, 'Distribution': True, 'Model type': True, 'Embedding size': True},
)
fig.update_layout(legend=legend_dict)
fig.update_traces(marker=marker_dict)
fig.add_trace(ppl_fit)
fig.data = fig.data[::-1]
fig.write_html('plots/vocab.html')

In [8]:
fig = px.scatter(
    df,
    x='Entropy',
    y='Test set perplexity',
    title='Test set perplexity vs. entropy',
    labels={'x': 'Entropy', 'y': 'Test set perplexity', 'color': 'Uni- or bigram and distribution'},
    color=combine_attrs(uni_or_bi, dist),
    hover_data={'Vocab size': True, 'Softmax': True, 'Model type': True, 'Embedding size': True},
)
fig.update_layout(legend=legend_dict)
fig.update_traces(marker=marker_dict)
fig.add_trace(ppl_fit)
fig.data = fig.data[::-1]
fig.write_html('plots/uni_or_bi_dist.html')

In [9]:
fig = px.scatter(
    df,
    x='Entropy',
    y='Test set perplexity',
    title='Test set perplexity vs. entropy',
    labels={'x': 'Entropy', 'y': 'Test set perplexity', 'color': 'Model type'},
    color='Model type',
    hover_data={'Vocab size': True, 'Softmax': True, 'Uni- or bigram': True, 'Distribution': True, 'Embedding size': True},
)
fig.update_layout(legend=legend_dict)
fig.update_traces(marker=marker_dict)
fig.add_trace(ppl_fit)
fig.data = fig.data[::-1]
fig.write_html('plots/model_type.html')

In [10]:
fig = px.scatter(
    df,
    x='Entropy',
    y='Test set perplexity',
    title='Test set perplexity vs. entropy',
    labels={'x': 'Entropy', 'y': 'Test set perplexity', 'color': 'Embedding size'},
    color=[str(x) for x in embd_size],
    hover_data={'Vocab size': True, 'Softmax': True, 'Uni- or bigram': True, 'Distribution': True, 'Model type': True},
)
fig.update_layout(legend=legend_dict)
fig.update_traces(marker=marker_dict)
fig.add_trace(ppl_fit)
fig.data = fig.data[::-1]
fig.write_html('plots/embd_size.html')

In [11]:
fig = px.scatter(
    df,
    x='Entropy',
    y='Test set perplexity',
    title='Test set perplexity vs. entropy',
    labels={'x': 'Entropy', 'y': 'Test set perplexity', 'color': 'Softmax'},
    color='Softmax',
    hover_data={'Vocab size': True, 'Uni- or bigram': True, 'Distribution': True, 'Model type': True, 'Embedding size': True},
)
fig.update_layout(legend=legend_dict)
fig.update_traces(marker=marker_dict)
fig.add_trace(ppl_fit)
fig.data = fig.data[::-1]
fig.write_html('plots/softmax.html')

In [12]:
fig = px.scatter(
    df,
    x='Entropy',
    y='Test set average cross-entropy',
    title=f'Test set average cross-entropy vs. entropy, R^2={r_squared:.4f}',
    labels={'x': 'Entropy', 'y': 'Test set average cross-entropy', 'color': 'Uni- or bigram'},
    color='Uni- or bigram',
    hover_data={'Vocab size': True, 'Softmax': True, 'Distribution': True, 'Model type': True, 'Embedding size': True},
)
fig.update_layout(legend=legend_dict)
fig.update_traces(marker=marker_dict)
fig.add_trace(ce_fit)
fig.add_trace(y_equals_x)
fig.data = fig.data[::-1]
fig.write_html('plots/uni_or_bi_ce.html')

In [13]:
fig = px.scatter(
    df,
    x='Entropy',
    y='Test set average cross-entropy',
    title=f'Test set average cross-entropy vs. entropy, R^2={r_squared:.4f}',
    labels={'x': 'Entropy', 'y': 'Test set average cross-entropy', 'color': 'Vocab size'},
    color=[str(x) for x in vocab_size],
    category_orders={'color': reversed([str(x) for x in sorted(vocab_size)])},
    hover_data={'Uni- or bigram': True, 'Softmax': True, 'Distribution': True, 'Model type': True, 'Embedding size': True},
)
fig.update_layout(legend=legend_dict)
fig.update_traces(marker=marker_dict)
fig.add_trace(ce_fit)
fig.add_trace(y_equals_x)
fig.data = fig.data[::-1]
fig.write_html('plots/vocab_ce.html')

In [14]:
fig = px.scatter(
    df,
    x='Entropy',
    y='Test set average cross-entropy',
    title=f'Test set average cross-entropy vs. entropy, R^2={r_squared:.4f}',
    labels={'x': 'Entropy', 'y': 'Test set average cross-entropy', 'color': 'Uni- or bigram and distribution'},
    color=combine_attrs(uni_or_bi, dist),
    hover_data={'Vocab size': True, 'Softmax': True, 'Model type': True, 'Embedding size': True},
)
fig.update_layout(legend=legend_dict)
fig.update_traces(marker=marker_dict)
fig.add_trace(ce_fit)
fig.add_trace(y_equals_x)
fig.data = fig.data[::-1]
fig.write_html('plots/uni_or_bi_dist_ce.html')

In [15]:
fig = px.scatter(
    df,
    x='Entropy',
    y='Test set average cross-entropy',
    title=f'Test set average cross-entropy vs. entropy, R^2={r_squared:.4f}',
    labels={'x': 'Entropy', 'y': 'Test set average cross-entropy', 'color': 'Model type'},
    color='Model type',
    hover_data={'Vocab size': True, 'Softmax': True, 'Uni- or bigram': True, 'Distribution': True, 'Embedding size': True},
)
fig.update_layout(legend=legend_dict)
fig.update_traces(marker=marker_dict)
fig.add_trace(ce_fit)
fig.add_trace(y_equals_x)
fig.data = fig.data[::-1]
fig.write_html('plots/model_type_ce.html')

In [16]:
fig = px.scatter(
    df,
    x='Entropy',
    y='Test set average cross-entropy',
    title=f'Test set average cross-entropy vs. entropy, R^2={r_squared:.4f}',
    labels={'x': 'Entropy', 'y': 'Test set average cross-entropy', 'color': 'Embedding size'},
    color=[str(x) for x in embd_size],
    hover_data={'Vocab size': True, 'Softmax': True, 'Uni- or bigram': True, 'Distribution': True, 'Model type': True},
)
fig.update_layout(legend=legend_dict)
fig.update_traces(marker=marker_dict)
fig.add_trace(ce_fit)
fig.add_trace(y_equals_x)
fig.data = fig.data[::-1]
fig.write_html('plots/embd_size_ce.html')

In [17]:
fig = px.scatter(
    df,
    x='Entropy',
    y='Test set average cross-entropy',
    title=f'Test set average cross-entropy vs. entropy, R^2={r_squared:.4f}',
    labels={'x': 'Entropy', 'y': 'Test set average cross-entropy', 'color': 'Softmax'},
    color='Softmax',
    hover_data={'Vocab size': True, 'Embedding size': True, 'Uni- or bigram': True, 'Distribution': True, 'Model type': True},
)
fig.update_layout(legend=legend_dict)
fig.update_traces(marker=marker_dict)
fig.add_trace(ce_fit)
fig.add_trace(y_equals_x)
fig.data = fig.data[::-1]
fig.write_html('plots/softmax_ce.html')

In [18]:
# for i, j in enumerate(jsons):
#     plt.figure(figsize=(10, 1))
#     data = j['losses']
#     plt.scatter(range(len(data)), data, s=0.1)
#     plt.title(f'{j["settings"]}')
#     plt.xticks(range(0, len(data)+1, 2000), labels=[f'{int(x/1000)}K' for x in range(0, len(data)+1, 2000)])
#     plt.xlabel('Training steps')
#     plt.ylabel('Cross-entropy')
#     plt.show()